In [612]:
#import keras
import nltk
import pandas as pd
import numpy as np
import re
import codecs
import gc
from nltk.corpus import wordnet
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pivottablejs import pivot_ui

gc.collect()
    

12521

In [613]:
inputFile="../data/cleaned/outputFile 0-500000.csv"
df = pd.read_csv(inputFile)
df = df.drop(['id'], axis=1)
df=df[0:500000]

# our categories and their related words
classes=['unisex', 'men', 'women', 'kid', 'baby']
manNet=['man', 'male', 'gentleman', 'guy', 'gent','masculine',' manlike', ' mannish', 'Amazonian']
womanNet=['woman', 'women', 'lady', 'female' 'ladies', 'girl','feminine','unmacho','metrosexual' ]
babyNet= ['baby','toddler','infant', 'babe', 'bambino', 'infant', 'neonate', 'newborn']
kidNet=['kid','child', 'children', 'child', 'youth', 'joni','schoolchild', 'schoolgirl', 'schoolkid','junior']
unisexNet=['unisex','androgynous', 'genderless', 'unisexual']




In [614]:
# Helper Functions

def plot_class_distribution (data_frame, groupby_feature,class_name, starting_index=0):
    grouped  = data_frame.groupby([class_name]) 
    values=grouped[groupby_feature].agg(np.size)[starting_index:]  
    labels =  values.index.tolist()  
    y_pos = np.arange(len(labels))
    plt.bar(y_pos, values)
    plt.xticks(y_pos, labels)
    plt.xlabel('Product categories')
    plt.ylabel('Number of Products')
    plt.show()



In [615]:
#Cleaning the dataset


#map NLTK’s POS tags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


#Lemmatize Normalization
def normalize (tokens):
    lem = WordNetLemmatizer()
    return [lem.lemmatize(token,pos=get_wordnet_pos(token)) for token in tokens]

#clean up  tokenized  data
def standardize_tokens (tokens):
    return [token.lower() for token in tokens]

#Few regular expressions to clean up  text data
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r".com", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.replace(r"nan", "")
    df[text_field] = df[text_field].str.lower()
    #tokenize text
    df[text_field] = df[text_field].apply(word_tokenize)
    #remove stop words
    stop_words=set(stopwords.words("english"))
    df[text_field] = df[text_field].apply(lambda x: [item for item in x if item not in stop_words] )
    #normalize text
    df[text_field] = df[text_field].apply(normalize)
        
   


In [616]:

df['title']=df['title'].astype(str)
df['product_type']=df['product_type'].astype(str)
df['description']=df['description'].astype(str)
df['store_domain']=df['store_domain'].astype(str)
df['vendor_name']=df['vendor_name'].astype(str)

 
df['all_text_original'] = df.values.sum(axis=1)
#df['all_text_original'] = df.apply( str.join( " " ))
df['vendor_name_original'] = df['vendor_name'].str.lower()

standardize_text(df, 'product_type')
standardize_text(df, 'vendor_name')
standardize_text(df, 'title')
standardize_text(df, 'store_domain')
standardize_text(df, 'description')


#combine all token in all columns into one column
df['all_tokens'] = df.values[:,0:5].sum(axis=1)


    

In [617]:
df=df[['product_type', 'vendor_name', 'title', 'store_domain', 'description',
        'all_tokens','vendor_name_original','all_text_original']]
display(df.head())

product_type vendor_name  \
0            [cookware]    [hestan]   
1            [cookware]    [hestan]   
2             [skillet]    [hestan]   
3  [saucepan, sauciers]    [hestan]   
4            [cookware]    [hestan]   

                                               title       store_domain  \
0  [hestan, nanobond, stainless, 3qt, 2, 8l, cove...  [atlantagrilpany]   
1   [hestan, nanobond, titanium, stockpot, 8, quart]  [atlantagrilpany]   
2        [hestan, nanobond, 8, 5, '', open, skillet]         [dasallas]   
3  [hestan, nanobond, 5, quart, cover, essential,...         [dasallas]   
4  [hestan, probond, forge, stainless, steel, ult...  [atlantagrilpany]   

  description                                         all_tokens  \
0          []  [cookware, hestan, hestan, nanobond, stainless...   
1          []  [cookware, hestan, hestan, nanobond, titanium,...   
2          []  [skillet, hestan, hestan, nanobond, 8, 5, '', ...   
3          []  [saucepan, sauciers, hestan, hestan, nanobond,...   
4          []  [cookware, hestan, hestan, probond, forge, sta...   

  vendor_name_original                                  all_text_original  
0               hestan  CookwareHestanHestan Nanobond Stainless 3qt/2....  
1               hestan  CookwareHestanHestan Nanobond Titanium Stockpo...  
2               hestan  SkilletsHestanHestan Nanobond 8.5" Open Skille...  
3               hestan  Saucepans & SauciersHestanHestan Nanobond 5-Qu...  
4               hestan  CookwareHestanHestan ProBond Forged Stainless ...

##  Frequency of the categories and their synonyms in the product information 


In [618]:
#count occurence of keyword in the list
def countFreq (product_info, keywordList):
    total_count =0
    for item in product_info:
      total_count += keywordList.count(item)
    return total_count


        

df['unisex'] = df['all_tokens'].apply(countFreq,keywordList=unisexNet)   
df['men'] = df['all_tokens'].apply(countFreq,keywordList=manNet) 
df['women'] = df['all_tokens'].apply(countFreq,keywordList=womanNet)  
df['baby'] = df['all_tokens'].apply(countFreq,keywordList=babyNet)  
df['kid'] = df['all_tokens'].apply(countFreq,keywordList=kidNet)
  
df['class']=  '-1'

display(df.head())

product_type vendor_name  \
0            [cookware]    [hestan]   
1            [cookware]    [hestan]   
2             [skillet]    [hestan]   
3  [saucepan, sauciers]    [hestan]   
4            [cookware]    [hestan]   

                                               title       store_domain  \
0  [hestan, nanobond, stainless, 3qt, 2, 8l, cove...  [atlantagrilpany]   
1   [hestan, nanobond, titanium, stockpot, 8, quart]  [atlantagrilpany]   
2        [hestan, nanobond, 8, 5, '', open, skillet]         [dasallas]   
3  [hestan, nanobond, 5, quart, cover, essential,...         [dasallas]   
4  [hestan, probond, forge, stainless, steel, ult...  [atlantagrilpany]   

  description                                         all_tokens  \
0          []  [cookware, hestan, hestan, nanobond, stainless...   
1          []  [cookware, hestan, hestan, nanobond, titanium,...   
2          []  [skillet, hestan, hestan, nanobond, 8, 5, '', ...   
3          []  [saucepan, sauciers, hestan, hestan, nanobond,...   
4          []  [cookware, hestan, hestan, probond, forge, sta...   

  vendor_name_original                                  all_text_original  \
0               hestan  CookwareHestanHestan Nanobond Stainless 3qt/2....   
1               hestan  CookwareHestanHestan Nanobond Titanium Stockpo...   
2               hestan  SkilletsHestanHestan Nanobond 8.5" Open Skille...   
3               hestan  Saucepans & SauciersHestanHestan Nanobond 5-Qu...   
4               hestan  CookwareHestanHestan ProBond Forged Stainless ...   

   unisex  men  women  baby  kid class  
0       0    0      0     0    0    -1  
1       0    0      0     0    0    -1  
2       0    0      0     0    0    -1  
3       0    0      0     0    0    -1  
4       0    0      0     0    0    -1

## Choose the label with the highest occurance of the keyword 

In [619]:
# choose the label with the highest occurance of the keyword 
def findLabel(dataFrame): 
        maxCount=max(dataFrame)
        if maxCount> 0:
            maxLabel= dataFrame[dataFrame == maxCount].index[0] 
        else:
            maxLabel='-1'
        return maxLabel
    
df.loc[:,'class']=df.loc[:,classes].apply(findLabel,axis=1) 
display (df.head())

product_type vendor_name  \
0            [cookware]    [hestan]   
1            [cookware]    [hestan]   
2             [skillet]    [hestan]   
3  [saucepan, sauciers]    [hestan]   
4            [cookware]    [hestan]   

                                               title       store_domain  \
0  [hestan, nanobond, stainless, 3qt, 2, 8l, cove...  [atlantagrilpany]   
1   [hestan, nanobond, titanium, stockpot, 8, quart]  [atlantagrilpany]   
2        [hestan, nanobond, 8, 5, '', open, skillet]         [dasallas]   
3  [hestan, nanobond, 5, quart, cover, essential,...         [dasallas]   
4  [hestan, probond, forge, stainless, steel, ult...  [atlantagrilpany]   

  description                                         all_tokens  \
0          []  [cookware, hestan, hestan, nanobond, stainless...   
1          []  [cookware, hestan, hestan, nanobond, titanium,...   
2          []  [skillet, hestan, hestan, nanobond, 8, 5, '', ...   
3          []  [saucepan, sauciers, hestan, hestan, nanobond,...   
4          []  [cookware, hestan, hestan, probond, forge, sta...   

  vendor_name_original                                  all_text_original  \
0               hestan  CookwareHestanHestan Nanobond Stainless 3qt/2....   
1               hestan  CookwareHestanHestan Nanobond Titanium Stockpo...   
2               hestan  SkilletsHestanHestan Nanobond 8.5" Open Skille...   
3               hestan  Saucepans & SauciersHestanHestan Nanobond 5-Qu...   
4               hestan  CookwareHestanHestan ProBond Forged Stainless ...   

   unisex  men  women  baby  kid class  
0       0    0      0     0    0    -1  
1       0    0      0     0    0    -1  
2       0    0      0     0    0    -1  
3       0    0      0     0    0    -1  
4       0    0      0     0    0    -1

## The distribution of classes including Nan

In [620]:
grouped  = df.groupby(['class'])     
print(grouped['product_type'].agg(np.size))

class
-1        455684
baby       10344
kid         5467
men         2925
unisex      2475
women      23105
Name: product_type, dtype: int64


In [621]:
plot_class_distribution (df, 'product_type','class', starting_index=0)

## The distribution of classes excluding Nan

In [622]:
plot_class_distribution (df, 'product_type','class', starting_index=1)

## Find the most common keywords in product information and label records containg those keywords


###  1) Define commmon keywords for each category

In [623]:
unisexProduct= ['electronics', 'phone','fruit','movie','vegetable', 'seafood','ipad','video','music','book', 'dairy',
                'egg', 'fridge', 'phone', 'supplement', 'cable', 'cookware', 'cook', 'novel', 'bike','headphone',
               'appliance','battery', 'vitamin','fence', 'garden','speaker','camera', 'kitchen', 'radio', 'backpack'
                'frozen', 'food', 'household', 'safety', 'skate' ]

womanProduct=['jewellery', 'pregnancy', 'make up', 'nail polish', 'eye shadow', 'skirt','Manicure', 'Pedicure'   ]

menProduct= [  'shave' , 'belt', 'TUXEDO',  ] 
# suit but not  jumpsuit, maybe tie but not'tiered',

kidProduct=['school', 'disney', 'spider', 'barbie','doll'  ]

babyProduct=['Pacifier' ,'Strollers', 'diapers', 'potty', 'walkers', 'playmat', 
             'Car Seat', 'lip liner', 'Babyliss', 'maternity', 
             'Teether', 'nursery', 'carrier', 'crib', 'Rattle', 'sleeper']

#lemmitize and standardize the all the categories lists
unisexProduct=normalize(unisexProduct)
unisexProduct=standardize_tokens(unisexProduct)
menProduct=normalize(menProduct)
menProduct=standardize_tokens(menProduct)
womanProduct=normalize(womanProduct)
womanProduct=standardize_tokens(womanProduct)
kidProduct=normalize(kidProduct)
kidProduct=standardize_tokens(kidProduct) 
babyProduct=normalize(babyProduct)
babyProduct=standardize_tokens(babyProduct) 

all_categories_lists=[unisexProduct,   menProduct ,  womanProduct ,  kidProduct ,  babyProduct]


### 2) Find most commmon words in product information which are not included in the categories lists

####   Let's inspect word and vocabulary of our data set

In [624]:
#combine all rows' tokens  into one list
all_words=list([a for b in df['all_tokens'].tolist() for a in b])
all_words=list(filter(lambda a: a not in [',', '(', ')'], all_words))


VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))

5030788 words total, with a vocabulary size of 115749


In [625]:
#Find the most commmon words in product_type
from nltk.probability import FreqDist
fdist = FreqDist(all_words)
print(fdist)
fdist.most_common(10)

# Frequency Distribution Plot
import matplotlib.pyplot as plt
fdist.plot(40,cumulative=False)
plt.show()

<FreqDist with 115749 samples and 5030788 outcomes>


In [626]:
#Find the most frequent words which are not included  categories lists
from nltk.probability import FreqDist
fdist = FreqDist(all_words)
 
for word,number in fdist.most_common(30):
    if word not in all_categories_lists:
        print (word)
        #pass
    

co
's
black
au
shirt
top
set
uk
michael
''
dress
woman
accessory
kor
new
pearl
white
necklace
kate
earring
spade
blue
bag
print
gold
ring
copper
2
men
leather


### 3) Label records containg common keywords

In [627]:
#find the total frequency of a list of keywords in a tokenized list 
def count_occurance_keyword ( tokenized_list, category_list ) :
        count=0
        text_data= ' '.join( tokenized_list)  
        for keyword in category_list:
                count += text_data.count(keyword+" ")       
        return count     
    
def findLabel_commonKeywords(dataFrame, feature): 
    count_unisex = count_occurance_keyword(dataFrame[feature], unisexProduct) 
    count_men   =  count_occurance_keyword(dataFrame[feature], menProduct) 
    count_woman  = count_occurance_keyword(dataFrame[feature], womanProduct) 
    count_kid    = count_occurance_keyword(dataFrame[feature], kidProduct) 
    count_baby   = count_occurance_keyword(dataFrame[feature], babyProduct) 
    
    index=['unisex', 'men', 'women', 'kid', 'baby']
    counters=[count_unisex, count_men, count_woman,count_kid,count_baby]
    frequency= pd.Series(counters, index=index)
    
    # find label with maximum frequency
    max_frequency= max(frequency)
    max_label= frequency.idxmax() if max_frequency> 0 else '-1' 
    return max_label


#notLabled = df[ df['class'] == '-1' ]  
not_labled_index=  df['class'] == '-1' 
df.loc[not_labled_index, 'class'] = df.loc[not_labled_index,:].apply(findLabel_commonKeywords, axis=1,args=['all_tokens']) 


In [628]:
display(  df[ df['class'] != '-1' ].head() )


product_type vendor_name  \
0       [cookware]    [hestan]   
1       [cookware]    [hestan]   
4       [cookware]    [hestan]   
5       [cookware]    [hestan]   
6  [cookware, set]    [hestan]   

                                               title       store_domain  \
0  [hestan, nanobond, stainless, 3qt, 2, 8l, cove...  [atlantagrilpany]   
1   [hestan, nanobond, titanium, stockpot, 8, quart]  [atlantagrilpany]   
4  [hestan, probond, forge, stainless, steel, ult...  [atlantagrilpany]   
5  [hestan, probond, forge, stainless, steel, ess...  [atlantagrilpany]   
6        [hestan, nanobond, 5, piece, set, cookware]         [dasallas]   

  description                                         all_tokens  \
0          []  [cookware, hestan, hestan, nanobond, stainless...   
1          []  [cookware, hestan, hestan, nanobond, titanium,...   
4          []  [cookware, hestan, hestan, probond, forge, sta...   
5          []  [cookware, hestan, hestan, probond, forge, sta...   
6          []  [cookware, set, hestan, hestan, nanobond, 5, p...   

  vendor_name_original                                  all_text_original  \
0               hestan  CookwareHestanHestan Nanobond Stainless 3qt/2....   
1               hestan  CookwareHestanHestan Nanobond Titanium Stockpo...   
4               hestan  CookwareHestanHestan ProBond Forged Stainless ...   
5               hestan  CookwareHestanHestan ProBond Forged Stainless ...   
6               hestan  Cookware SetsHestanHestan Nanobond 5-Piece Set...   

   unisex  men  women  baby  kid   class  
0       0    0      0     0    0  unisex  
1       0    0      0     0    0  unisex  
4       0    0      0     0    0  unisex  
5       0    0      0     0    0  unisex  
6       0    0      0     0    0  unisex

In [629]:
plot_class_distribution (df, 'product_type','class', starting_index=1)

In [630]:

print(df['class'].value_counts())

-1        411773
unisex     32079
women      29487
baby       12959
kid         8359
men         5343
Name: class, dtype: int64


## Labeling more records based on vendor names

If products from a vendor all belong to one particular category (given that at least 10 products are listed), we can assign that category to other products from the same vendor

In [631]:
#pivot_ui(labeled_data )

In [632]:
homo_brands= {}
labeled_data=df[df['class']!= '-1'].copy()
grouped  = labeled_data.groupby(['vendor_name_original'])
for key, group in grouped:
   class_group = grouped.get_group(key).groupby(['class']) 
   #print(key, len(class_group ), class_group['class'].count() )
   # if all products belong to one category
   if len(class_group)==1:
      # If at least 10 products are listed for a company
      if  (class_group['class'].count()[0]) > 10 :
         homo_brands[key]= list(class_group.groups.keys())[0]

print(homo_brands)

{'a newborn nest': 'baby', 'aanyalinen': 'women', 'addis': 'unisex', 'aikyou': 'women', 'amber noir': 'men', 'ann taylor loft': 'women', 'anna beck': 'women', 'ans performance': 'unisex', 'anson belt & buckle': 'men', "arden's garden 1": 'unisex', 'aromatique': 'unisex', 'arturia': 'unisex', 'atypical man': 'men', 'aubrey jane boutique': 'women', 'aykasa': 'baby', 'babbi shoes': 'women', 'baby banana': 'baby', 'baby banana brush': 'baby', 'bambinos clothing co.': 'baby', 'be awara juniors': 'kid', 'begg & co': 'unisex', 'begg and co': 'unisex', 'bella ballou': 'unisex', 'blackwood': 'unisex', 'bold raw': 'unisex', 'boody wear': 'women', 'booja booja': 'unisex', 'borner': 'unisex', 'brand: springer': 'unisex', 'brenda riddle designs': 'unisex', 'bridal brilliance': 'women', 'bridgestone': 'unisex', 'brooklyn soap company': 'men', 'bulk reef supply': 'unisex', 'bullet cable': 'unisex', 'burrow baby': 'baby', 'by johnny.': 'women', 'cabbage white': 'women', 'cariuma': 'women', 'carpe diem

In [633]:
homo_vendor_bool=  df['vendor_name_original'].apply( lambda x: x in list(homo_brands.keys())) 
not_labled_bool=   df['class'] == '-1'
#records which are not labeled yet and belong to homo vendor
homo_notLabeld_bool=np.logical_and(not_labled_bool, homo_vendor_bool)
homo_notLabeld_index= df[homo_notLabeld_bool].index

pd.DataFrame({'homo_vendor': homo_vendor_bool, 'not_labled_bool': not_labled_bool,'homo_notLabeld_bool':homo_notLabeld_bool  })
display(df) 

product_type        vendor_name  \
0                  [cookware]           [hestan]   
1                  [cookware]           [hestan]   
2                   [skillet]           [hestan]   
3        [saucepan, sauciers]           [hestan]   
4                  [cookware]           [hestan]   
...                       ...                ...   
499995                 [part]  [zev, technology]   
499996      [firearm, pistol]  [zev, technology]   
499997     [grip, pad, stock]  [zev, technology]   
499998  [barrel, choke, tube]  [zev, technology]   
499999              [outdoor]  [zev, technology]   

                                                    title  \
0       [hestan, nanobond, stainless, 3qt, 2, 8l, cove...   
1        [hestan, nanobond, titanium, stockpot, 8, quart]   
2             [hestan, nanobond, 8, 5, '', open, skillet]   
3       [hestan, nanobond, 5, quart, cover, essential,...   
4       [hestan, probond, forge, stainless, steel, ult...   
...                                                   ...   
499995           [zev, technology, 3lbs, striker, spring]   
499996  [zev, technology, spartan, complete, slide, w,...   
499997           [zev, wedge, lock, lrg, hndgrd, rfl, 14]   
499998             [zev, barrel, g19, dimple, blk, thrdd]   
499999            [zev, strppd, exo, glk19, g3, rmr, blk]   

                   store_domain description  \
0             [atlantagrilpany]          []   
1             [atlantagrilpany]          []   
2                    [dasallas]          []   
3                    [dasallas]          []   
4             [atlantagrilpany]          []   
...                         ...         ...   
499995  [shop, blackboxcustoms]          []   
499996      [greystonetactical]          []   
499997      [gunslingersarmory]          []   
499998      [gunslingersarmory]          []   
499999     [yellowstartactical]          []   

                                               all_tokens  \
0       [cookware, hestan, hestan, nanobond, stainless...   
1       [cookware, hestan, hestan, nanobond, titanium,...   
2       [skillet, hestan, hestan, nanobond, 8, 5, '', ...   
3       [saucepan, sauciers, hestan, hestan, nanobond,...   
4       [cookware, hestan, hestan, probond, forge, sta...   
...                                                   ...   
499995  [part, zev, technology, zev, technology, 3lbs,...   
499996  [firearm, pistol, zev, technology, zev, techno...   
499997  [grip, pad, stock, zev, technology, zev, wedge...   
499998  [barrel, choke, tube, zev, technology, zev, ba...   
499999  [outdoor, zev, technology, zev, strppd, exo, g...   

       vendor_name_original  \
0                    hestan   
1                    hestan   
2                    hestan   
3                    hestan   
4                    hestan   
...                     ...   
499995     zev technologies   
499996     zev technologies   
499997     zev technologies   
499998     zev technologies   
499999     zev technologies   

                                        all_text_original  unisex  men  women  \
0       CookwareHestanHestan Nanobond Stainless 3qt/2....       0    0      0   
1       CookwareHestanHestan Nanobond Titanium Stockpo...       0    0      0   
2       SkilletsHestanHestan Nanobond 8.5" Open Skille...       0    0      0   
3       Saucepans & SauciersHestanHestan Nanobond 5-Qu...       0    0      0   
4       CookwareHestanHestan ProBond Forged Stainless ...       0    0      0   
...                                                   ...     ...  ...    ...   
499995  PartsZev TechnologiesZEV Technologies 3lbs Str...       0    0      0   
499996  Firearms - PistolZEV TechnologiesZEV Technolog...       0    0      0   
499997  Grips/Pads/StocksZEV TechnologiesZev Wedge Loc...       0    0      0   
499998  Barrels/Choke TubesZEV TechnologiesZev Barrel ...       0    0      0   
499999  OutdoorZEV TechnologiesZEV STRPPD EXO FOR GLK1...       0    0      0   

        baby  k

In [634]:
print("not labeled \n",df[not_labled_bool].index)
print("homo \n",df[homo_vendor_bool].index)
print("homo and notlabeled \n",df[homo_notLabeld_bool].index)


not labeled 
 Int64Index([     2,      3,      7,     10,     11,     12,     13,     14,
                20,     21,
            ...
            499990, 499991, 499992, 499993, 499994, 499995, 499996, 499997,
            499998, 499999],
           dtype='int64', length=411773)
homo 
 Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            495188, 495192, 495199, 495200, 495204, 495214, 495220, 495228,
            495244, 495255],
           dtype='int64', length=34702)
homo and notlabeled 
 Int64Index([     2,      3,      7,     10,     11,     12,     13,     14,
                20,     21,
            ...
            495183, 495188, 495192, 495200, 495204, 495214, 495220, 495228,
            495244, 495255],
           dtype='int64', length=21723)


In [635]:
print(df['class'].value_counts())

def get_homo_class ( x ):
    vendor=x['vendor_name_original']
    #print(vendor, homo_brands[vendor])
    return homo_brands[vendor]
    
df.loc[homo_notLabeld_index, 'class'] = df.loc[homo_notLabeld_index,:].apply(get_homo_class, axis=1)
print(df['class'].value_counts())
display(df)

-1        411773
unisex     32079
women      29487
baby       12959
kid         8359
men         5343
Name: class, dtype: int64
-1        390050
unisex     43374
women      36481
baby       13631
kid        10272
men         6192
Name: class, dtype: int64


product_type        vendor_name  \
0                  [cookware]           [hestan]   
1                  [cookware]           [hestan]   
2                   [skillet]           [hestan]   
3        [saucepan, sauciers]           [hestan]   
4                  [cookware]           [hestan]   
...                       ...                ...   
499995                 [part]  [zev, technology]   
499996      [firearm, pistol]  [zev, technology]   
499997     [grip, pad, stock]  [zev, technology]   
499998  [barrel, choke, tube]  [zev, technology]   
499999              [outdoor]  [zev, technology]   

                                                    title  \
0       [hestan, nanobond, stainless, 3qt, 2, 8l, cove...   
1        [hestan, nanobond, titanium, stockpot, 8, quart]   
2             [hestan, nanobond, 8, 5, '', open, skillet]   
3       [hestan, nanobond, 5, quart, cover, essential,...   
4       [hestan, probond, forge, stainless, steel, ult...   
...                                                   ...   
499995           [zev, technology, 3lbs, striker, spring]   
499996  [zev, technology, spartan, complete, slide, w,...   
499997           [zev, wedge, lock, lrg, hndgrd, rfl, 14]   
499998             [zev, barrel, g19, dimple, blk, thrdd]   
499999            [zev, strppd, exo, glk19, g3, rmr, blk]   

                   store_domain description  \
0             [atlantagrilpany]          []   
1             [atlantagrilpany]          []   
2                    [dasallas]          []   
3                    [dasallas]          []   
4             [atlantagrilpany]          []   
...                         ...         ...   
499995  [shop, blackboxcustoms]          []   
499996      [greystonetactical]          []   
499997      [gunslingersarmory]          []   
499998      [gunslingersarmory]          []   
499999     [yellowstartactical]          []   

                                               all_tokens  \
0       [cookware, hestan, hestan, nanobond, stainless...   
1       [cookware, hestan, hestan, nanobond, titanium,...   
2       [skillet, hestan, hestan, nanobond, 8, 5, '', ...   
3       [saucepan, sauciers, hestan, hestan, nanobond,...   
4       [cookware, hestan, hestan, probond, forge, sta...   
...                                                   ...   
499995  [part, zev, technology, zev, technology, 3lbs,...   
499996  [firearm, pistol, zev, technology, zev, techno...   
499997  [grip, pad, stock, zev, technology, zev, wedge...   
499998  [barrel, choke, tube, zev, technology, zev, ba...   
499999  [outdoor, zev, technology, zev, strppd, exo, g...   

       vendor_name_original  \
0                    hestan   
1                    hestan   
2                    hestan   
3                    hestan   
4                    hestan   
...                     ...   
499995     zev technologies   
499996     zev technologies   
499997     zev technologies   
499998     zev technologies   
499999     zev technologies   

                                        all_text_original  unisex  men  women  \
0       CookwareHestanHestan Nanobond Stainless 3qt/2....       0    0      0   
1       CookwareHestanHestan Nanobond Titanium Stockpo...       0    0      0   
2       SkilletsHestanHestan Nanobond 8.5" Open Skille...       0    0      0   
3       Saucepans & SauciersHestanHestan Nanobond 5-Qu...       0    0      0   
4       CookwareHestanHestan ProBond Forged Stainless ...       0    0      0   
...                                                   ...     ...  ...    ...   
499995  PartsZev TechnologiesZEV Technologies 3lbs Str...       0    0      0   
499996  Firearms - PistolZEV TechnologiesZEV Technolog...       0    0      0   
499997  Grips/Pads/StocksZEV TechnologiesZev Wedge Loc...       0    0      0   
499998  Barrels/Choke TubesZEV TechnologiesZev Barrel ...       0    0      0   
499999  OutdoorZEV TechnologiesZEV STRPPD EXO FOR GLK1...       0    0      0   

        baby  k

# Export Labeled Data

In [636]:
df['all_text']= df['all_tokens'].apply(lambda x: " ".join(x))
df.to_csv("../data/labeled/labeled_data.csv", index=True)
 


In [637]:
display(labeled_data.head())

product_type vendor_name  \
0       [cookware]    [hestan]   
1       [cookware]    [hestan]   
4       [cookware]    [hestan]   
5       [cookware]    [hestan]   
6  [cookware, set]    [hestan]   

                                               title       store_domain  \
0  [hestan, nanobond, stainless, 3qt, 2, 8l, cove...  [atlantagrilpany]   
1   [hestan, nanobond, titanium, stockpot, 8, quart]  [atlantagrilpany]   
4  [hestan, probond, forge, stainless, steel, ult...  [atlantagrilpany]   
5  [hestan, probond, forge, stainless, steel, ess...  [atlantagrilpany]   
6        [hestan, nanobond, 5, piece, set, cookware]         [dasallas]   

  description                                         all_tokens  \
0          []  [cookware, hestan, hestan, nanobond, stainless...   
1          []  [cookware, hestan, hestan, nanobond, titanium,...   
4          []  [cookware, hestan, hestan, probond, forge, sta...   
5          []  [cookware, hestan, hestan, probond, forge, sta...   
6          []  [cookware, set, hestan, hestan, nanobond, 5, p...   

  vendor_name_original                                  all_text_original  \
0               hestan  CookwareHestanHestan Nanobond Stainless 3qt/2....   
1               hestan  CookwareHestanHestan Nanobond Titanium Stockpo...   
4               hestan  CookwareHestanHestan ProBond Forged Stainless ...   
5               hestan  CookwareHestanHestan ProBond Forged Stainless ...   
6               hestan  Cookware SetsHestanHestan Nanobond 5-Piece Set...   

   unisex  men  women  baby  kid   class  
0       0    0      0     0    0  unisex  
1       0    0      0     0    0  unisex  
4       0    0      0     0    0  unisex  
5       0    0      0     0    0  unisex  
6       0    0      0     0    0  unisex